In [5]:
import pandas as pd

# --- Load and Clean Data ---
# --- This sample data is the correct form as it downloaded from the software as it contain many unwanted rows and columns.
file_path = 'Sample_data.csv'
# --- Skiprows = 3 is given as the first 3 rows contain just the headings
df = pd.read_csv(file_path, skiprows=3,dtype={'Phone2': str})
# --- Only wanted columns are taken 
df = df[['Bill Date', 'OUTLET NAME', 'Customer Name', 'Phone2']]
# ---  Skipping the first row after column name as it is a total of of orher columns
df = df.iloc[1:].copy()

# Clean phone numbers
df['Phone2'] = df['Phone2'].astype(str).str.replace(r'\D+', '', regex=True).str.strip()
# making every customer name capital to follow same pattern
df['Customer Name'] = df['Customer Name'].astype(str).str.strip().str.upper()



# --- Define Conditions ---
def apply_conditions(df):
    df['NOB'] = 1  # Every row is one bill
    # Repeting_More_3 : is to identify that if a outlet repeat a same customer number more that 3 times it is considered as a scam
    df['Repeting_More_3'] = (df['Phone2'] != '9091600900') & (df.groupby(['Bill Date', 'Phone2'])['Phone2'].transform('count') > 3)
    # F2D_Cust_num : Outlet are advised to give company Customer number if the customer refuse to give their number, so this classify the outlet that used customer care number
    df['F2D_Cust_num'] = (df['Customer Name'] == 'CUSTOMER_CARE') & (df['Phone2'] != '9091600900')
    # Fake number : Numbers which start rather than 6,7,8,9 and not 10 digits are considered fake number
    df['Fake number'] = (df['Phone2'].str.len() != 10) | (~df['Phone2'].str[0].isin(['6', '7', '8', '9']))
    # CustNum_WOT_F2D_name : Classifies number is customer care number but the customer name is another 'A customer care number only have same customer name 'customer_care'
    df['CustNum_WOT_F2D_name'] = (df['Phone2'] == '9091600900') & (df['Customer Name'] != 'CUSTOMER_CARE')
    # Fake_CustNum : Customer number written wrongly at the end
    df['Fake_CustNum'] = df['Phone2'].str.startswith('90916009') & (~df['Phone2'].str[8:10].eq('00'))
    return df

df = apply_conditions(df)

# --- Create Summary Table ---
summary = df.groupby(['Bill Date', 'OUTLET NAME']).agg(
    NOB=('NOB', 'sum'),
    Repeting_More_3 = ('Repeting_More_3', 'sum'),
    F2D_Cust_num=('F2D_Cust_num', 'sum'),
    Fake_number=('Fake number', 'sum'),
    CustNum_WOT_F2D_name=('CustNum_WOT_F2D_name', 'sum'),
    Fake_CustNum=('Fake_CustNum', 'sum'),
).reset_index()

# Calculate percentages
for col in ['Repeting_More_3', 'F2D_Cust_num', 'Fake_number', 'CustNum_WOT_F2D_name', 'Fake_CustNum']:
    summary[f'{col} %'] = (summary[col] / summary['NOB']).round(4)

# --- Add Grand Total Row ---
grand = {
    'Bill Date': '',
    'OUTLET NAME': 'Grand Total',
    'NOB': summary['NOB'].sum()
}
for col in ['Repeting_More_3', 'F2D_Cust_num', 'Fake_number', 'CustNum_WOT_F2D_name', 'Fake_CustNum']:
    grand[col] = summary[col].sum()
    grand[f'{col} %'] = round(grand[col] / grand['NOB'], 4)

summary = pd.concat([summary, pd.DataFrame([grand])], ignore_index=True)

# --- Add Remarks Column to Main Data ---
def remark_row(row):
    reasons = []
    if row['Repeting_More_3']:
        reasons.append('Repeting_More_3')
    if row['F2D_Cust_num']:
        reasons.append('F2D_Cust_num')
    if row['Fake number']:
        reasons.append('Fake number')
    if row['CustNum_WOT_F2D_name']:
        reasons.append('CustNum_WOT_F2D_name')
    if row['Fake_CustNum']:
        reasons.append('Fake_CustNum')
    if (row['Customer Name'].strip().upper() == 'customer_care') and (row['Phone2'] == '9091600900'):
        reasons.append('Customer_care_num')
    return ', '.join(reasons) if reasons else 'Actual Customer'

df['Remark'] = df.apply(remark_row, axis=1)

# --- Export to Excel ---
output_file = 'violation_phone_summary.xlsx'
with pd.ExcelWriter(output_file, engine='xlsxwriter') as writer:
    summary.to_excel(writer, sheet_name='Summary Report', index=False)
    df.to_excel(writer, sheet_name='Remarks', index=False)

    # Format percentage columns
    workbook = writer.book
    summary_ws = writer.sheets['Summary Report']
    percent_fmt = workbook.add_format({'num_format': '0.00%', 'align': 'center'})

    for col_idx, col in enumerate(summary.columns):
        if '%' in col:
            summary_ws.set_column(col_idx, col_idx, 18, percent_fmt)
        else:
            summary_ws.set_column(col_idx, col_idx, 18)

print(f"✅ Report saved to '{output_file}' with Summary and Remarks sheets.")


✅ Report saved to 'violation_phone_summary.xlsx' with Summary and Remarks sheets.
